## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

## 실행환경 설정

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## epochs, batch size 설정

In [3]:
EPOCHS = 300
BATCH_SIZE = 128

## 학습용 데이터, 테스트용 데이터셋 설정 

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=True,
                     download=True,
                     transform=transforms.Compose([
                         transforms.RandomCrop(32, padding=1),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                              (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=False,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                              (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [00:21<00:00, 7914313.60it/s]


Extracting ./.data\cifar-10-python.tar.gz to ./.data


## Residual 모듈 생성

In [25]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## 모델의 instance와 최적화 함수

In [26]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50,
                                      gamma=0.1)
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

## train함수와 evaluate 함수

In [27]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

## 학습

In [28]:
for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 1.5569, Accuracy: 42.81%
[2] Test Loss: 1.3820, Accuracy: 52.36%
[3] Test Loss: 0.8920, Accuracy: 69.23%
[4] Test Loss: 0.7483, Accuracy: 74.36%
[5] Test Loss: 0.8195, Accuracy: 72.56%
[6] Test Loss: 0.8053, Accuracy: 72.39%
[7] Test Loss: 0.8220, Accuracy: 72.67%
[8] Test Loss: 0.6518, Accuracy: 77.66%
[9] Test Loss: 0.6960, Accuracy: 76.51%
[10] Test Loss: 0.8773, Accuracy: 69.95%
[11] Test Loss: 0.7519, Accuracy: 74.38%
[12] Test Loss: 0.6601, Accuracy: 78.23%
[13] Test Loss: 0.5638, Accuracy: 80.70%
[14] Test Loss: 0.8563, Accuracy: 72.50%
[15] Test Loss: 0.6849, Accuracy: 77.09%
[16] Test Loss: 0.8570, Accuracy: 72.56%
[17] Test Loss: 0.6651, Accuracy: 77.15%
[18] Test Loss: 0.6114, Accuracy: 79.07%
[19] Test Loss: 0.7713, Accuracy: 74.61%
[20] Test Loss: 0.5977, Accuracy: 79.87%
[21] Test Loss: 0.7374, Accuracy: 74.29%
[22] Test Loss: 0.6232, Accuracy: 78.85%
[23] Test Loss: 0.7775, Accuracy: 75.26%
[24] Test Loss: 0.8395, Accuracy: 74.31%
[25] Test Loss: 0.7147, A

[199] Test Loss: 0.2888, Accuracy: 91.34%
[200] Test Loss: 0.2894, Accuracy: 91.49%
[201] Test Loss: 0.2876, Accuracy: 91.48%
[202] Test Loss: 0.2879, Accuracy: 91.47%
[203] Test Loss: 0.2884, Accuracy: 91.41%
[204] Test Loss: 0.2887, Accuracy: 91.41%
[205] Test Loss: 0.2891, Accuracy: 91.42%
[206] Test Loss: 0.2886, Accuracy: 91.38%
[207] Test Loss: 0.2888, Accuracy: 91.47%
[208] Test Loss: 0.2892, Accuracy: 91.46%
[209] Test Loss: 0.2909, Accuracy: 91.32%
[210] Test Loss: 0.2878, Accuracy: 91.48%
[211] Test Loss: 0.2886, Accuracy: 91.43%
[212] Test Loss: 0.2891, Accuracy: 91.43%
[213] Test Loss: 0.2891, Accuracy: 91.38%
[214] Test Loss: 0.2884, Accuracy: 91.32%
[215] Test Loss: 0.2878, Accuracy: 91.32%
[216] Test Loss: 0.2869, Accuracy: 91.56%
[217] Test Loss: 0.2881, Accuracy: 91.45%
[218] Test Loss: 0.2888, Accuracy: 91.45%
[219] Test Loss: 0.2877, Accuracy: 91.49%
[220] Test Loss: 0.2894, Accuracy: 91.47%
[221] Test Loss: 0.2896, Accuracy: 91.35%
[222] Test Loss: 0.2877, Accuracy: